In [1]:
from tscsRL.environments.TSCSEnv import DiscreteTSCSEnv, ContinuousTSCSEnv
from tscsRL.environments.GradientTSCSEnv import DiscreteGradientTSCSEnv, ContinuousGradientTSCSEnv
from tscsRL.environments.RadiiTSCSEnv import DiscreteRadiiTSCSEnv, ContinuousRadiiTSCSEnv
from tscsRL.agents import ddqn, ddpg
from tscsRL import utils
import imageio
import torch

In [2]:
name = 'ddqnRadii-test'
path = 'results/' + name
env_params = utils.jsonToDict(path + '/env_params.json')
agent_params = utils.jsonToDict(path + '/agent_params.json')

In [9]:
env = DiscreteRadiiTSCSEnv(
    kMax=env_params['kMax'],
    kMin=env_params['kMin'],
    nFreq=env_params['nFreq'])

agent = ddqn.DDQNAgent(
    env.observation_space,
    env.action_space,
    agent_params,
    name)
agent.epsilon = 0.05

In [10]:
agent.load_checkpoint(path + '/checkpoints/', 3400)

In [12]:
## Starting from a random config
state = env.reset()
## End starting from random config

## Starting from a predefined config
# env.config = torch.tensor([[ 4.9073,  0.9211,  2.3434, -4.5528,  2.6180,  4.6988, -2.3804,  1.6361]])
# env.counter = torch.tensor([[0.0]])
# env.setMetric(env.config)

env.info['initial'] = env.RMS.item()
env.info['lowest'] = env.info['initial']
env.info['final'] = None
env.info['score'] = 0
state = env.getState()
## End starting from random config

done = False

results = {
    'config': [],
    'rms': [],
    'tscs': []}

while not done:
    results['config'].append(env.config)
    results['rms'].append(env.RMS)
    results['tscs'].append(env.TSCS)

    action = agent.select_action(state)
    nextState, reward, done, info = env.step(action)

#     print(reward, done)
    state = nextState

## Initial stuff
initialConfig = results['config'][0]
initialRMS = results['rms'][0]
initialTSCS = results['tscs'][0]

## Optimal stuff
minIdx = results['rms'].index(min(results['rms']))
optimalConfig = results['config'][minIdx]
optimalRMS = results['rms'][minIdx]
optimalTSCS = results['tscs'][minIdx]

print('RESULTS:')
print(f'Initial config: {initialConfig}')
print(f'Initial RMS: {initialRMS}')
print(f'Initial TSCS: {initialTSCS}')
print()
print(f'Min config: {optimalConfig}')
print(f'Min rms: {optimalRMS}')
print(f'Min tscs: {optimalTSCS}')

RESULTS:
Initial config: tensor([[ 3.5000,  0.0000],
        [ 2.6812,  2.2498],
        [ 0.6078,  3.4468],
        [-1.7500,  3.0311],
        [-3.2889,  1.1971],
        [-3.2889, -1.1971],
        [-1.7500, -3.0311],
        [ 0.6078, -3.4468],
        [ 2.6812, -2.2498]], dtype=torch.float64)
Initial RMS: tensor([[11.5263]])
Initial TSCS: tensor([[ 4.3174,  5.9410, 34.2762, 12.0271,  6.3756,  4.2726,  3.2210,  2.6081,
          2.2180,  1.9561,  1.7744]])

Min config: tensor([[ 3.5000,  0.0000],
        [ 2.6812,  2.2498],
        [ 0.6078,  3.4468],
        [-1.7500,  3.0311],
        [-3.2889,  1.1971],
        [-3.2889, -1.1971],
        [-1.7500, -3.0311],
        [ 0.6078, -3.4468],
        [ 2.6812, -2.2498]], dtype=torch.float64)
Min rms: tensor([[4.2483]])
Min tscs: tensor([[ 2.1084,  2.4444, 11.9761,  4.7053,  2.9824,  2.1895,  1.7325,  1.4400,
          1.2421,  1.1042,  1.0070]])
